In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import warnings
warnings.filterwarnings('ignore')

import os, sys
currentdir = os.path.dirname(os.path.abspath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
from helper_functions import *

from datetime import timedelta
import os
import investpy
import yfinance as yf
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# import eikon as ek
# ek.set_app_key('9a249e0411184cf49e553b61a6e76c52d295ec17')

def get_data_alpha(ticker):

    if ticker == "GOLDBEES.NS":
        ticker_inv = "GBES"
    if ticker == "^NSEI":
        ticker_inv = "NSEI"

    temp_og = get_data(ticker, "yfinance", "india")

    if ticker == "GOLDBEES.NS":
        temp_og = temp_og[temp_og["Close"] < 100]

    today_data = yf.download(ticker, start=str(date.today() - timedelta(days=1)), interval="1m")
    if len(today_data)==0:
        today_time_close = temp_og.iloc[-1]["Date"]
    else:
        today_data.reset_index(inplace=True)
        today_data.drop(['Adj Close'], axis=1, inplace=True)
        if ticker == "GOLDBEES.NS":
            today_data = today_data.loc[today_data["Close"] > 1]

        today_time_close = today_data.iloc[-1]["Datetime"]

        temp_og = pd.concat([temp_og, pd.DataFrame([{"Date": pd.Timestamp(year=today_data.iloc[-1]["Datetime"].year,
                                                                          month=today_data.iloc[-1]["Datetime"].month,
                                                                          day=today_data.iloc[-1]["Datetime"].day),\
                                                     "Open": today_data.iloc[-1]["Open"],
                                                     "Close": today_data.iloc[-1]["Close"],
                                                     "High": today_data.iloc[-1]["High"],
                                                     "Low": today_data.iloc[-1]["Low"],
                                                     "Volume": today_data.iloc[-1]["Volume"]}])],axis=0).reset_index().drop(['index'], axis=1)
        temp_og.drop_duplicates(subset="Date",
                            keep='first', inplace=True)
    temp_og = add_fisher(temp_og)
    return temp_og, today_time_close

def select_all_strategies(train_monthsf, datesf, temp_ogf, ticker, save=True):
    inputs =[]
    for date_i in range(len(datesf)-(int(train_monthsf/3)+1)):
        inputs.append([date_i, datesf, temp_ogf, train_monthsf])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results = pool.map(get_strategies_brute_force, inputs)
    finally: # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    res_test_update = [pd.DataFrame(columns=["Lookback", "Low Bound", "High Bound", "AvgWinLoss",\
                                       "Optimization_Years"])] * (len(datesf)-(int(24/3)+1))

    for i in range(len(results)):
        res_test_update[results[i][0]+int((train_monthsf-24)/3)] = pd.concat([res_test_update[results[i][0]],results[i][1].reset_index().drop(['index'], axis=1)], axis=0)

    if save==True:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_{int(train_monthsf/12)}_All_Strategies.pkl','rb') as file:
            res_test = pickle.load(file)
        res_test.append(res_test_update[-1])
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_{int(train_monthsf/12)}_All_Strategies.pkl', 'wb') as file:
            pickle.dump(res_test, file)
    return res_test

def select_strategies_from_corr_filter(res_testf2,res_testf4,res_testf8, datesf, temp_ogf, num_opt_periodsf,num_strategiesf, ticker, save=True):
    train_monthsf = 24  #minimum optimization lookback
    res_total = [None]*(len(datesf)-(int(train_monthsf/3)+1))
    for i in range(len(datesf)-(int(train_monthsf/3)+1)):
        if num_opt_periodsf==1:
            res_total[i] = pd.concat([res_testf2[i]], axis = 0)
        if num_opt_periodsf==2:
            res_total[i] = pd.concat([res_testf2[i],res_testf4[i]], axis=0)
        if num_opt_periodsf==3:
            res_total[i] = pd.concat([res_testf2[i],res_testf4[i],res_testf8[i]], axis=0)
        res_total[i] = res_total[i].reset_index().drop(['index'], axis=1)

    ss_test_update = [None]*(len(datesf)-(int(train_monthsf/3)+1))
    res_test_update = [None]*(len(datesf)-(int(train_monthsf/3)+1))
    inputs = []
    for date_i in range(len(datesf)-(int(train_monthsf/3)+1)):
        inputs.append([date_i, datesf, temp_ogf,res_total, num_strategiesf,train_monthsf])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_filtered = pool.map(corr_sortino_filter, inputs)
    finally: # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    for i in range(len(datesf)-(int(train_monthsf/3)+1)):
        ss_test_update[results_filtered[i][0]] = results_filtered[i][1]
        res_test_update[results_filtered[i][0]] = results_filtered[i][2]

    if save==True:
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_ss.pkl','rb') as file:
            ss_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_res.pkl','rb') as file:
            res_test = pickle.load(file)

        ss_test.append(ss_test_update[-1])
        res_test.append(res_test_update[-1])

        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_ss.pkl', 'wb') as file:
            pickle.dump(ss_test, file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_res.pkl', 'wb') as file:
            pickle.dump(res_test, file)

    return ss_test, res_test

def SendMail(subject_text_add, text_restricted,text_nifty, text_gold, text, printdf_nifty, printdf_gold, assets, ImgFileNameList):
    msg = MIMEMultipart()
    msg['Subject'] = f'{subject_text_add} [ADITYA INDIVIDUAL] Strategy Update on NIFTY50 Dual Momentum - Alpha & Accelerating Momentum'
    msg['From'] = 'algo_notifications@acsysindia.com'
    msg['Cc'] = 'suprabhashsahu@acsysindia.com' #, aditya@shankar.biz
    msg['To'] = 'algo_notifications@acsysindia.com'

    for ImgFileName in ImgFileNameList:
        with open(ImgFileName, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(ImgFileName))
        msg.attach(image)

    text_restricted = MIMEText(text_restricted)
    msg.attach(text_restricted)

    text_nifty = MIMEText(text_nifty)
    msg.attach(text_nifty)

    strategies_nifty = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(printdf_nifty.to_html())

    part1 = MIMEText(strategies_nifty, 'html')
    msg.attach(part1)

    text_gold = MIMEText(text_gold)
    msg.attach(text_gold)

    strategies_gold = """\
        <html>
          <head></head>
          <body>
            {0}
          </body>
        </html>
        """.format(printdf_gold.to_html())

    part2 = MIMEText(strategies_gold, 'html')
    msg.attach(part2)

    text = MIMEText(text)
    msg.attach(text)

    assetsprint = """\
            <html>
              <head></head>
              <body>
                {0}
              </body>
            </html>
            """.format(assets.to_html())

    part3 = MIMEText(assetsprint, 'html')
    msg.attach(part3)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login('algo_notifications@acsysindia.com', 'esahYah8')
    s.sendmail('algo_notifications@acsysindia.com', ['suprabhashsahu@acsysindia.com', 'algo_notifications@acsysindia.com'], msg.as_string())  #, 'aditya@shankar.biz'
    s.quit()

def signal_print(inp, ticker):
    if inp == 0:
        signal = f"Neutral on {ticker}, Long on Fixed Income"
    else:
        signal = f"Long on {ticker}, Neutral on Fixed Income"
    return signal

def execute_nifty(ticker, number_of_optimization_periods,recalib_months,num_strategies,metric):
    # Download data
    temp_og, today_time_close = get_data_alpha(ticker)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10], end="2024-06-15",
                                 freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months, dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    temp_res['Return'] = np.log(temp_res['Close'] / temp_res['Close'].shift(1))
    temp_res['Market_Return'] = np.exp(temp_res['Return'].expanding().sum())-1
    temp_res['Strategy_Return'] = np.exp(temp_res['S_Return'].expanding().sum())-1
    temp_res['Portfolio Value'] = ((temp_res['Strategy_Return'] + 1) * 10000)
    temp_res.reset_index(inplace=True)
    temp_res_nifty = temp_res.copy()
    temp_res_nifty.columns = ["Date"] + [column + "_nifty" for column in temp_res_nifty.columns if column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title('Strategy Backtest: Nifty Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    text = ""
    text = text + "*"*100 + "\n"
    text = text + "NIFTY Alpha" + "\n"
    text = text + "Phase-1: Top strategies are ones which maximises the Average Win/Average Loss of the trades made over the lookback period" + "\n"
    text = text + "Top Strategies are selected based on lookbacks of 24 months" + "\n"
    text = text + f"Top Strategies are selected every {recalib_months} months" + "\n"
    text = text + f"Top {num_strategies} strategies are selected" + "\n"
    text = text + f"Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months = 24))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"Phase-2: Weights of the selected strategies are calculated such that rolling outperformance against benchmark is maximised over a lookback of 24 months "+ "\n"
    text = text + f"Last Recalibrated Nifty Alpha on {str(dates[-2])[:11]}" + "\n"
    text = text + f"Recalibrating Nifty Alpha on {str(dates[-2] + relativedelta(months = recalib_months))[:11]}" + "\n" + "\n"
    text = text + "Alpha Stats for last 252 trading days:" + "\n"
    text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n" + "\n"
    text = text + f"Signal at : {str(today_time_close)[:19]}: {signal_print(temp_res.iloc[-1]['signal'],ticker)}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'],ticker)}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'],ticker)}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Portfolio Value'], 2)}" + "\n" + "\n"
    text = text + "Selected Strategies: " + "\n" + "\n"
    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_nifty, text

def execute_gold(ticker, number_of_optimization_periods,recalib_months,num_strategies,metric):

    # Download data
    temp_og, today_time_close = get_data_alpha(ticker)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10],
                                 end="2024-06-15", freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months,
             dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    temp_res['Return'] = np.log(temp_res['Close'] / temp_res['Close'].shift(1))
    temp_res['Market_Return'] = np.exp(temp_res['Return'].expanding().sum())-1
    temp_res['Strategy_Return'] = np.exp(temp_res['S_Return'].expanding().sum())-1
    temp_res['Portfolio Value'] = ((temp_res['Strategy_Return'] + 1) * 10000)
    temp_res.reset_index(inplace=True)
    temp_res_gold = temp_res.copy()
    temp_res_gold.columns = ["Date"] + [column + "_gold" for column in temp_res_gold.columns if
                                        column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title('Strategy Backtest: Gold Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    text = ""
    text = text + "*" * 100 + "\n"
    text = text + "GOLD Alpha" + "\n"
    text = text + "Phase-1: Top strategies are ones which maximises the Average Win/Average Loss of the trades made over the lookback period" + "\n"
    text = text + "Top Strategies are selected based on lookbacks of 24 and 48 months" + "\n"
    text = text + f"Top Strategies are selected every {recalib_months} months" + "\n"
    text = text + f"Top {num_strategies} strategies are selected" + "\n"
    text = text + f"24-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=24))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"48-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=48))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"Phase-2: Weights of the selected strategies are calculated such that rolling outperformance against benchmark is maximised over a lookback of 24 months " + "\n"
    text = text + f"Last Recalibrated Gold Alpha on {str(dates[-2])[:11]}" + "\n"
    text = text + f"Recalibrating Gold Alpha on {str(dates[-2] + relativedelta(months = recalib_months))[:11]}" + "\n" + "\n"
    text = text + "Alpha Stats for last 252 trading days:" + "\n"
    text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n" + "\n"
    text = text + f"Signal at : {str(today_time_close)[:19]} : {signal_print(temp_res.iloc[-1]['signal'],ticker)}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'],ticker)}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'],ticker)}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Portfolio Value'], 2)}" + "\n" + "\n"
    text = text + "Selected Strategies: " + "\n" + "\n"
    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_gold, text


In [3]:

ticker_nifty = "^NSEI"
number_of_optimization_periods_nifty = 1
recalib_months_nifty = 3
num_strategies_nifty = 5
metric_nifty = 'outperformance'

ticker_gold = "GOLDBEES.NS"
number_of_optimization_periods_gold = 2
recalib_months_gold = 6
num_strategies_gold = 1
metric_gold = 'outperformance'

text = ""
print(f"Executing: {datetime.now()}")

text_restricted = "Restricted List is : [PGHH, PFIZER]" + "\n"

index = ".BSESN"
with open(f'BSE_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
tickers = [ticker[:-3] for ticker in tickers]
# Restricted Stock
#tickers.remove('AXBK')

recalibrating_months = 1
top_nassets = 10
training_period = 24  # 24/48/96
dates_recalibrating = valid_dates(
    pd.date_range(start="2009-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))


Executing: 2021-12-20 14:04:46.845417


In [4]:

data_inp = prepare_portfolio_data(tickers, recalibrating_months, "datatables", "")


In [5]:

data_inp = data_inp[data_inp['Date'] != '2020-11-14']
data_inp = data_inp[data_inp['Date'] != '2020-11-17']

with open(f'BSE_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)

temp_og_nifty, signal_time_nifty, dates_all_ss_nifty, dates_ss_nifty, printdf_nifty, temp_res_nifty, text_nifty = execute_nifty(ticker_nifty,number_of_optimization_periods_nifty,recalib_months_nifty,num_strategies_nifty,metric_nifty)
temp_og_gold, signal_time_gold, dates_all_ss_gold, dates_ss_gold, printdf_gold, temp_res_gold, text_gold = execute_gold(ticker_gold,number_of_optimization_periods_gold,recalib_months_gold,num_strategies_gold,metric_gold)

if temp_res_nifty.iloc[-1]['signal_nifty']==1:
    subject_text_add = "[Long on Index]"
if (temp_res_nifty.iloc[-1]['signal_nifty']==0) & (temp_res_gold.iloc[-1]['signal_gold']==1):
    subject_text_add = "[Long on Gold]"
if (temp_res_nifty.iloc[-1]['signal_nifty']==0) & (temp_res_gold.iloc[-1]['signal_gold']==0):
    subject_text_add = "[Long on Cash]"

data_inp_backtest = pd.concat([data_inp.set_index('Date'), temp_res_nifty.set_index('Date'), temp_res_gold.set_index('Date')], axis=1, join='inner').reset_index()

rebalancing_months = 13
dates_rebalancing = []
assetsb = []
for date_i in range(len(dates_recalibrating) - (int(training_period / recalibrating_months) + 1)):
    if ((recalibrating_months * date_i) % rebalancing_months == 0) & (
            dates_recalibrating[date_i + int(training_period / recalibrating_months)] >=
            data_inp_backtest["Date"][0]):
        dates_rebalancing.append(dates_recalibrating[date_i + int(training_period / recalibrating_months)])
        assetsb.append(assets[date_i])
dates_rebalancing.append(date.today()+timedelta(1))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Importing selected strategies: 2021-12-20 14:05:10.739102
Selected Strategies for Testing period beginning: 2021-09-30 00:00:00 and ending: 2021-12-21
   Lookback  Low Bound  High Bound  AvgWinLoss  Optimization_Years
0       230        2.0         1.5     1608.67                 2.0
1       130        2.8         1.8     1027.50                 2.0
2       250        2.8         2.0      813.00                 2.0
3        90        0.0        -0.2      314.38                 2.0
4        50        4.5         2.0      231.67                 2.0
5        50        3.2         1.5      131.33                 2.0
6       310        1.8         2.8      121.62                 2.0
7       330        1.5         2.5       76.18                 2.0
8        50        0.8         2.2       69.71                 2.0
9        50        0.0         0.5       

<Figure size 1080x360 with 0 Axes>

In [6]:
data_inp_backtest[["Date", "ASPN"]]

,Date,ASPN
0,2012-01-31,299.008818
1,2012-02-01,293.009275
2,2012-02-02,296.283568
3,2012-02-03,294.516648
4,2012-02-06,298.839114
...,...,...
2408,2021-12-13,3279.850000
2409,2021-12-14,3297.500000
2410,2021-12-15,3301.950000
2411,2021-12-16,3298.600000


In [7]:

data_inp = data_inp_backtest.copy()


In [16]:
data_inp_backtest[["Date", "ICBK"]]

,Date,ICBK
0,2012-01-31,NaN
1,2012-02-01,NaN
2,2012-02-02,NaN
3,2012-02-03,NaN
4,2012-02-06,NaN
...,...,...
2408,2021-12-13,754.00
2409,2021-12-14,758.65
2410,2021-12-15,752.25
2411,2021-12-16,741.15


In [10]:


current_balance = 7930747.142346616
gold_allocation = 0
nifty_allocation = 0
cash_allocation = 0
portfolio_value = pd.DataFrame()

for date_i in [1]:  #range(len(dates_rebalancing) - 1)
    try:
        test = data_inp.loc[
            (data_inp["Date"] >= str(dates_rebalancing[date_i])) & (
                    data_inp["Date"] < str(dates_rebalancing[date_i + 1]))].reset_index().drop(['index'], axis=1)
        test.set_index(test["Date"], inplace=True)
    except:
        test = data_inp.loc[(data_inp["Date"] >= str(dates_rebalancing[date_i]))]
        test.set_index(test["Date"], inplace=True)
    tickers = assetsb[date_i]["Ticker"].to_list()

    percent_tracker_current_balance_ticker = {}
    percent_tracker_units_ticker = {}
    percent_ticker = {}
    for ticker in tickers:
        percent_tracker_current_balance_ticker[ticker] = current_balance / len(tickers)
        percent_tracker_units_ticker[ticker] = percent_tracker_current_balance_ticker[ticker] / test.iloc[0][ticker]

    current_balance_ticker = {}
    units_ticker = {}
    for ticker in tickers:
        current_balance_ticker[ticker] = current_balance / len(tickers)
        units_ticker[ticker] = current_balance_ticker[ticker] / test.iloc[0][ticker]

    print(units_ticker)
    break

    units_gold = 0

    for i in range(len(test)):

        for ticker in tickers:
            percent_tracker_current_balance_ticker[ticker] = percent_tracker_units_ticker[ticker] *  test.iloc[i][ticker]
        for ticker in tickers:
            percent_ticker[ticker] = percent_tracker_current_balance_ticker[ticker] / sum(percent_tracker_current_balance_ticker.values())

        signal_nifty = test["signal_nifty"].iloc[i]
        signal_gold = test["signal_gold"].iloc[i]

        if signal_nifty == 1:
            nifty_allocation = current_balance
            gold_allocation = 0
            cash_allocation = 0
        if (signal_nifty == 0) & (signal_gold == 1):
            nifty_allocation = 0
            gold_allocation = current_balance / 2
            cash_allocation = current_balance / 2
        if (signal_nifty == 0) & (signal_gold == 0):
            nifty_allocation = 0
            gold_allocation = 0
            cash_allocation = current_balance

        if ((test["signal_nifty"].iloc[i] == 1) & (test["signal_nifty"].shift(1).fillna(0).iloc[i] == 0)):
            for ticker in tickers:
                current_balance_ticker[ticker] = nifty_allocation * percent_ticker[ticker]
                units_ticker[ticker] = current_balance_ticker[ticker] / test.iloc[i][ticker]

        if ((test["signal_nifty"].iloc[i] == 0) & (test["signal_nifty"].shift(1).fillna(1).iloc[i] == 1)):
            for ticker in tickers:
                current_balance_ticker[ticker] = 0
                units_ticker[ticker] = 0
            if signal_gold == 1:
                units_gold = gold_allocation / test.iloc[i]["Close_gold"]

        if ((test["signal_gold"].iloc[i] == 1) & (test["signal_gold"].shift(1).fillna(0).iloc[i] == 0)):
            units_gold = gold_allocation / test.iloc[i]["Close_gold"]

        if ((test["signal_gold"].iloc[i] == 0) & (test["signal_gold"].shift(1).fillna(1).iloc[i] == 1)):
            units_gold = 0

        if signal_nifty == 1:
            units_gold = 0
            nifty_allocation = 0
            for ticker in tickers:
                current_balance_ticker[ticker] = units_ticker[ticker] * test.iloc[i][ticker]
                nifty_allocation = nifty_allocation + current_balance_ticker[ticker]
        if (signal_nifty == 0) & (signal_gold == 1):
            gold_allocation = units_gold * test.iloc[i]["Close_gold"]
        cash_allocation = cash_allocation * (1 + 6 / 25200)
        current_balance = nifty_allocation + gold_allocation + cash_allocation

        if math.isnan(current_balance):
            print(date_i, i)

        portfolio_day = {'Date': test.iloc[i]["Date"], 'signal_nifty': signal_nifty, 'signal_gold':signal_gold,'units_gold':units_gold,'nifty_allocation':nifty_allocation,'gold_allocation':gold_allocation,'cash_allocation':cash_allocation,'Pvalue': current_balance}
        portfolio_day[f"Gold_close"] = test.iloc[i]["Close_gold"]
        for ticker in tickers:
            portfolio_day[f"{ticker}_close"] = test.iloc[i][ticker]
            portfolio_day[f"{ticker}_percent"] = percent_ticker[ticker]
            portfolio_day[f"{ticker}_units"] = units_ticker[ticker]
            portfolio_day[f"{ticker}_current_balance"] = current_balance_ticker[ticker]
        portfolio_day = pd.DataFrame([portfolio_day])
        portfolio_day = portfolio_day.set_index("Date")
        portfolio_value = pd.concat([portfolio_value, portfolio_day], axis=0, join="outer")

{'TCS': nan, 'NEST': 174.20360718609783, 'SUN': 1951.8235753021881, 'TAMO': 3019.1408774798174, 'REDY': 434.3472885890035, 'INFY': 2183.9245017246535, 'HCLT': 4098.870072322466, 'WIPR': 5583.698267324975, 'BAJA': 445.2224298179204, 'ICBK': nan}


In [13]:
get_data_investpy(symbol="ICBK", country="india", from_date="01/07/2007",to_date=(date.today()+timedelta(1)).strftime("%d/%m/%Y"))

,Open,High,Low,Close,Volume
Date,,,,,
2007-07-02,178.45,178.45,171.82,172.76,3542055
2007-07-03,173.27,177.26,171.93,175.76,8102682
2007-07-04,177.27,180.73,174.28,179.26,8392994
2007-07-05,180.18,183.45,176.38,182.48,18789858
2007-07-06,181.82,183.64,177.66,178.45,28157484
...,...,...,...,...,...
2021-12-14,751.05,764.35,746.30,758.65,17488100
2021-12-15,757.60,759.50,750.90,752.25,7358700
2021-12-16,757.90,757.90,738.95,741.15,14123200


In [21]:
portfolio_value

,signal_nifty,signal_gold,units_gold,nifty_allocation,gold_allocation,cash_allocation,Pvalue,Gold_close,BAJA_close,BAJA_percent,...,TEML_units,TEML_current_balance,ULTC_close,ULTC_percent,ULTC_units,ULTC_current_balance,INFY_close,INFY_percent,INFY_units,INFY_current_balance
Date,,,,,,,,,,,,,,,,,,,,,
2012-02-29,0,1,144062.550161,0.0,3.965374e+06,3.966318e+06,7.931691e+06,27.525360,1801.60,NaN,...,0.0,0.0,1406.40,NaN,0.0,0.0,355.649006,NaN,0.0,0.0
2012-03-01,0,1,144062.550161,0.0,3.832546e+06,3.966790e+06,7.799336e+06,26.603350,1774.20,NaN,...,0.0,0.0,1416.85,NaN,0.0,0.0,352.343454,NaN,0.0,0.0
2012-03-02,0,1,144062.550161,0.0,3.833045e+06,3.900597e+06,7.733641e+06,26.606810,1749.95,NaN,...,0.0,0.0,1424.55,NaN,0.0,0.0,350.992865,NaN,0.0,0.0
2012-03-05,0,1,144062.550161,0.0,3.829302e+06,3.867741e+06,7.697044e+06,26.580830,1701.90,NaN,...,0.0,0.0,1390.25,NaN,0.0,0.0,346.571073,NaN,0.0,0.0
2012-03-06,0,1,144062.550161,0.0,3.818355e+06,3.849438e+06,7.667793e+06,26.504841,1703.95,NaN,...,0.0,0.0,1405.60,NaN,0.0,0.0,351.825420,NaN,0.0,0.0
2012-03-07,0,1,144062.550161,0.0,3.852083e+06,3.834809e+06,7.686892e+06,26.738960,1733.20,NaN,...,0.0,0.0,1422.70,NaN,0.0,0.0,355.698343,NaN,0.0,0.0
2012-03-09,0,1,144062.550161,0.0,3.848671e+06,3.844361e+06,7.693033e+06,26.715281,1752.85,NaN,...,0.0,0.0,1472.70,NaN,0.0,0.0,352.176943,NaN,0.0,0.0
2012-03-12,0,1,144062.550161,0.0,3.856009e+06,3.847432e+06,7.703441e+06,26.766211,1787.75,NaN,...,0.0,0.0,1476.05,NaN,0.0,0.0,347.502301,NaN,0.0,0.0
2012-03-13,0,1,144062.550161,0.0,3.824858e+06,3.852637e+06,7.677495e+06,26.549980,1782.95,NaN,...,0.0,0.0,1500.50,NaN,0.0,0.0,352.731980,NaN,0.0,0.0


In [19]:

results_final, unit_ticker, unit_gold = backtest_Alpha_AM(dates_rebalancing, data_inp_backtest, assetsb)


In [20]:
results_final

,signal_nifty,signal_gold,units_gold,nifty_allocation,gold_allocation,cash_allocation,Pvalue,Gold_close,BAJA_close,BAJA_percent,...,COAL_units,COAL_current_balance,HDFC_close,HDFC_percent,HDFC_units,HDFC_current_balance,NTPC_close,NTPC_percent,NTPC_units,NTPC_current_balance
Date,,,,,,,,,,,,,,,,,,,,,
2012-02-29,0,1,144062.550161,0.0,3.965374e+06,3.966318e+06,7.931691e+06,27.525360,1801.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-01,0,1,144062.550161,0.0,3.832546e+06,3.966790e+06,7.799336e+06,26.603350,1774.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-02,0,1,144062.550161,0.0,3.833045e+06,3.900597e+06,7.733641e+06,26.606810,1749.95,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-05,0,1,144062.550161,0.0,3.829302e+06,3.867741e+06,7.697044e+06,26.580830,1701.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-06,0,1,144062.550161,0.0,3.818355e+06,3.849438e+06,7.667793e+06,26.504841,1703.95,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13,0,0,0.000000,0.0,0.000000e+00,NaN,NaN,41.549999,3305.45,NaN,...,NaN,NaN,2771.85,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2021-12-14,0,0,0.000000,0.0,0.000000e+00,NaN,NaN,41.549999,3291.65,NaN,...,NaN,NaN,2751.95,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2021-12-15,0,0,0.000000,0.0,0.000000e+00,NaN,NaN,41.470001,3294.50,NaN,...,NaN,NaN,2721.80,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [ ]:

                bench = yf.download("^BSESN", start='2007-01-01', end=str(date.today()+timedelta(1)))
                bench = bench.loc[bench["Close"] > 1]
                bench["Return"] = np.log(bench["Close"] / bench["Close"].shift(1))

                results_final["S_Return"] = pd.DataFrame(np.log(results_final["Pvalue"] / results_final["Pvalue"].shift(1)))
                temp_res = pd.concat([results_final, bench["Return"]], join="inner", axis=1)
                temp_res['Market_Return'] = np.exp(temp_res['Return'].expanding().sum())-1
                temp_res['Strategy_Return'] = np.exp(temp_res['S_Return'].expanding().sum())-1
                temp_res['Portfolio Value'] = temp_res['Pvalue']
                temp_res = temp_res.reset_index().rename(columns={'index': "Date"})

                plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
                plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
                plt.title('Strategy Backtest: Dual Momentum')
                plt.legend(loc=0)
                plt.tight_layout()
                plt.savefig("PerformanceNIFTY.jpg")
                plt.clf()

                text = text + "*" * 100+ "\n"
                text = text + "DUAL MOMENTUM STRATEGY"+ "\n"
                text = text + f"Recalibrated every {rebalancing_months} months, with training data of {training_period-recalibrating_months} months with a forward return  of {recalibrating_months} month"+ "\n"
                text = text + f"The number of assets being selected are: {top_nassets}"
                text = text + f"Last Recalibrated Accelerating Momentum on {str(dates_rebalancing[-2])[:11]}" + "\n"
                text = text + f"Recalibrating Accelerating Momentum on {str(dates_rebalancing[-2] + relativedelta(months = rebalancing_months))[:11]}" + "\n" + "\n"
                text = text + f"Strategies selected based on training data from: {str(dates_rebalancing[-2]- relativedelta(months = training_period))[:11]} to: {str(dates_rebalancing[-2] - relativedelta(months = recalibrating_months))[:11]} are selected" + "\n"
                text = text + f"Units and Momentum Values were calculated on {str(dates_rebalancing[-2])[:11]}" + "\n"
                text = text + "On Rebalancing Day, if Nifty Alpha is long, 100% of the Portolfio are allocated to the stocks below.\nIf Nifty Alpha is Neutral and Gold Alpha is Long, 50% of the Portfolio is allocated to GOLDBEES.NS and 50% to Fixed Income.\nIf both Alphas are Neutral, 100% of the Portfolio are allocated to Fixed Income" + "\n"+ "\n"
                text = text + "Stats for last 252 trading days:" + "\n"
                text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling Sharpe: {np.round(backtest_rolling_sharpe(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling Outperformance: {np.round(backtest_outperformance(temp_res[-252:], 0, 0), 2)}"
                text = text + "\n" + "\n" + "Overall Performance:" + "\n"
                text = text + f" A Portfolio of Rs 0.74072 Cr invested on: {str(dates_rebalancing[0])[:11]} is now valued at: {np.round(temp_res.iloc[-1]['Portfolio Value']/10000000, 4)} Cr." + "\n" + "\n"
                text = text + "Selected Assets: " + "\n"
                text = text + f"Number of units of GOLDBEES.NS: {unit_gold}"+ "\n"
                # text = f"Signal at : {str(today_time_close)[:19]} : {signal_print(temp_res.iloc[-1]['signal'])}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'])}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'])}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Portfolio Value'], 2)}" + "\n" + "\n" + text

                # if running_for_the_first_time == True:
                #     pass
                # else:
                SendMail(subject_text_add, text_restricted,text_nifty, text_gold, text, printdf_nifty, printdf_gold, pd.concat([assetsb[-1][["Ticker", "Accelerating Momentum"]].set_index("Ticker"), pd.DataFrame([unit_ticker]).transpose().rename(columns={0:"Units"})], axis=1), ["Performance^NSEI.jpg", "PerformanceGOLDBEES.NS.jpg", "PerformanceNIFTY.jpg"])

            # #Recalibrating Nifty Alpha
            # if (pd.to_datetime(date.today()) in dates_all_ss_nifty):
            #     print(f"Recalibrating Nifty at {datetime.now()}")
            #     res_test2 = select_all_strategies(24,dates_ss_nifty, temp_og_nifty, ticker_nifty,save=True)
            #     # res_test4 = select_all_strategies(48, dates_ss, temp_og, ticker, save=True)
            #     # res_test8 = select_all_strategies(96, dates_ss, temp_og, ticker, save=True)
            #     ss_test_imp, res_test_imp = select_strategies_from_corr_filter(res_test2,0,0, dates_ss_nifty, temp_og_nifty, number_of_optimization_periods_nifty,10, ticker_nifty, save=True)
            #
            #     res_test = []
            #     ss_test = []
            #     dates = []
            #     for date_i in range(len(dates_ss_nifty) - (int(24 / 3) + 1)):
            #         if (3 * date_i) % recalib_months_nifty == 0:
            #             dates.append(dates_ss_nifty[date_i + int(24 / 3)])
            #             ss_test.append(ss_test_imp[date_i])
            #             res_test.append(res_test_imp[date_i])
            #
            #     print(f"Recalibrating Weights: {datetime.now()}")
            #     inputs = []
            #     for date_i in range(len(dates)-1):
            #         inputs.append([date_i, dates, temp_og_nifty, ss_test, res_test, num_strategies_nifty, metric_nifty, recalib_months_nifty,dates_ss_nifty])
            #     try:
            #         pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
            #         weights_all = pool.map(optimize_weights_live, inputs)
            #     finally: # To make sure processes are closed in the end, even if errors happen
            #         pool.close()
            #         pool.join()
            #
            #     weights_update = [None]*(len(dates)-1)
            #     for date_i in range(len(dates)-1):
            #         weights_update[weights_all[date_i][0]] = weights_all[date_i][1]
            #
            #     with open(f'{ticker_nifty}/weights/Results_Ticker{ticker_nifty}_LP{number_of_optimization_periods_nifty}_Recal{recalib_months_nifty}_NS{num_strategies_nifty}_M{metric_nifty}.pkl','rb') as file:
            #         weights = pickle.load(file)
            #
            #     weights.append(weights_update[-1])
            #
            #     with open(f'{ticker_nifty}/weights/Results_Ticker{ticker_nifty}_LP{number_of_optimization_periods_nifty}_Recal{recalib_months_nifty}_NS{num_strategies_nifty}_M{metric_nifty}.pkl', 'wb') as file:
            #         pickle.dump(weights, file)
            #
            #     print(f"Recalibration for Nifty Over at: {datetime.now()}")
            #
            # # Recalibrating Gold Alpha
            # if (pd.to_datetime(date.today()) in dates_all_ss_gold):
            #     print(f"Recalibrating Gold at {datetime.now()}")
            #     res_test2 = select_all_strategies(24, dates_ss_gold, temp_og_gold, ticker_gold, save=True)
            #     res_test4 = select_all_strategies(48, dates_ss_gold, temp_og_gold, ticker_gold, save=True)
            #     # res_test8 = select_all_strategies(96, dates_ss, temp_og, ticker, save=True)
            #     ss_test_imp, res_test_imp = select_strategies_from_corr_filter(res_test2, res_test4, 0, dates_ss_gold,
            #                                                                    temp_og_gold,
            #                                                                    number_of_optimization_periods_gold,
            #                                                                    10, ticker_gold, save=True)
            #
            #     res_test = []
            #     ss_test = []
            #     dates = []
            #     for date_i in range(len(dates_ss_gold) - (int(24 / 3) + 1)):
            #         if (3 * date_i) % recalib_months_gold == 0:
            #             dates.append(dates_ss_gold[date_i + int(24 / 3)])
            #             ss_test.append(ss_test_imp[date_i])
            #             res_test.append(res_test_imp[date_i])
            #
            #     print(f"Recalibrating Weights: {datetime.now()}")
            #     inputs = []
            #     for date_i in range(len(dates) - 1):
            #         inputs.append(
            #             [date_i, dates, temp_og_gold, ss_test, res_test, num_strategies_gold, metric_gold,
            #              recalib_months_gold, dates_ss_gold])
            #     try:
            #         pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
            #         weights_all = pool.map(optimize_weights_live, inputs)
            #     finally:  # To make sure processes are closed in the end, even if errors happen
            #         pool.close()
            #         pool.join()
            #
            #     weights_update = [None] * (len(dates) - 1)
            #     for date_i in range(len(dates) - 1):
            #         weights_update[weights_all[date_i][0]] = weights_all[date_i][1]
            #
            #     with open(
            #             f'{ticker_gold}/weights/Results_Ticker{ticker_gold}_LP{number_of_optimization_periods_gold}_Recal{recalib_months_gold}_NS{num_strategies_gold}_M{metric_gold}.pkl',
            #             'rb') as file:
            #         weights = pickle.load(file)
            #
            #     weights.append(weights_update[-1])
            #
            #     with open(
            #             f'{ticker_gold}/weights/Results_Ticker{ticker_gold}_LP{number_of_optimization_periods_gold}_Recal{recalib_months_gold}_NS{num_strategies_gold}_M{metric_gold}.pkl',
            #             'wb') as file:
            #         pickle.dump(weights, file)
            #
            #     print(f"Recalibration for Gold Over at: {datetime.now()}")
            #
            # #Recalibrating AM Constituents
            # if (pd.to_datetime(date.today()) in dates_recalibrating):
            #     print(f"Start Recalibrating constituents of BSE at {datetime.now()}")
            #     index = ".BSESN"
            #     constituents = get_constituents(index)
            #
            #     with open(f'BSE_Constituents.pkl', 'wb') as file:
            #         pickle.dump(constituents, file)
            #
            #     print(f"End Recalibrating constituents of BSE at {datetime.now()}")
            #
            # #Recalibrating AM
            # if (pd.to_datetime(date.today()) in dates_recalibrating):
            #     print(f"Start Recalibrating AM of BSE at {datetime.now()}")
            #     assets.append(get_weights_stocks_live(constituents, top_nassets, recalibrating_months, training_period,dates_recalibrating, data_inp, save=False)[0])
            #     with open(f'BSE_RecalibPeriod_{int(1)}.pkl', 'wb') as file:
            #         pickle.dump(assets, file)
            #     print(f"End Recalibrating AM of BSE at {datetime.now()}")

        if running_for_the_first_time:
            pass
        else:
            if datetime.now()<datetime.now().replace(hour=run_hour).replace(minute=run_minute).replace(second=30):
                continue

        running_for_the_first_time = False

        print(f"Sleeping: {datetime.now()}")

        time_now = datetime.now()
        next_run = datetime.now()
        try:
            if (datetime.now().hour<run_hour) & (datetime.now().minute<run_minute):
                next_run = next_run.replace(day=next_run.day).replace(hour=run_hour).replace(minute=run_minute).replace(second=00)
            else:
                next_run = next_run.replace(day=next_run.day + 1).replace(hour=run_hour).replace(minute=run_minute).replace(second=00)
        except:
            if next_run.month == 12:
                next_run = next_run.replace(day=1).replace(month=1).replace(year=next_run.year + 1).replace(hour=run_hour).replace(minute=run_minute).replace(second=00)
            else:
                next_run = next_run.replace(day=1).replace(month=next_run.month + 1).replace(hour=run_hour).replace(minute=run_minute).replace(second=00)

        print(f"Supposed to wake up at: {datetime.now() + timedelta(seconds=(next_run - time_now).seconds - 150)}")
        time.sleep((next_run - time_now).seconds-150)
        print(f"Woken Up: {datetime.now()}")





